In [2]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *

In [4]:
probD = Table()
probD.read_data('Difficult.csv')

probI = Table()
probI.read_data('Intelligent.csv')

probG = Table()
probG.read_data('Grade.csv')

probS = Table()
probS.read_data('SAT.csv')

probL = Table()
probL.read_data('Letter.csv')

graph = UGraphModel()
graph.add("Difficult", probD)
graph.add("Intelligent", probI)
graph.add("Grade", probG)
graph.add("SAT", probS)
graph.add("Letter", probL)
graph.connect("Difficult", "Grade")
graph.connect("Intelligent", "Grade")
graph.connect("Intelligent", "SAT")
graph.connect("Grade", "Letter")
graph.println()

Vertices:
 V(Difficult, in:1, out:1)
0.6 easy 
0.4 hard 

 V(Intelligent, in:2, out:2)
0.7 low 
0.3 high 

 V(Grade, in:3, out:3)
0.3 ['low', 'easy', 'A'] 
0.4 ['low', 'easy', 'B'] 
0.3 ['low', 'easy', 'C'] 
0.05 ['low', 'hard', 'A'] 
0.25 ['low', 'hard', 'B'] 
0.7 ['low', 'hard', 'C'] 
0.9 ['high', 'easy', 'A'] 
0.08 ['high', 'easy', 'B'] 
0.02 ['high', 'easy', 'C'] 
0.5 ['high', 'hard', 'A'] 
0.3 ['high', 'hard', 'B'] 
0.2 ['high', 'hard', 'C'] 

 V(SAT, in:1, out:1)
0.95 ['low', 'low'] 
0.05 ['low', 'high'] 
0.2 ['high', 'low'] 
0.8 ['high', 'high'] 

 V(Letter, in:1, out:1)
0.1 ['A', 'weak'] 
0.9 ['A', 'strong'] 
0.4 ['B', 'weak'] 
0.6 ['B', 'strong'] 
0.99 ['C', 'weak'] 
0.01 ['C', 'strong'] 

------------------------------------------------------
Edges:
 E(Difficult, Grade, 0)
 E(Intelligent, Grade, 0)
 E(Intelligent, SAT, 0)
 E(Grade, Difficult, 0)
 E(Grade, Intelligent, 0)
 E(Grade, Letter, 0)
 E(SAT, Intelligent, 0)
 E(Letter, Grade, 0)



In [24]:
a = pd.read_csv("./Data/Grade.csv", header = 0)
df = pd.DataFrame(a)
df.loc['Difficult',:]

KeyError: 'Difficult'

In [8]:
grapD = DGraphModel()
for idx in range(10):
    grapD.add(str(idx))

grapD.connect('0', '1')
grapD.connect('0', '5')
grapD.connect('1', '7')
grapD.connect('3', '2')
grapD.connect('3', '4')
grapD.connect('3', '7')
grapD.connect('3', '8')
grapD.connect('4', '8')
grapD.connect('6', '0')
grapD.connect('6', '1')
grapD.connect('6', '2')
grapD.connect('8', '2')
grapD.connect('8', '7')
grapD.connect('9', '4')
grapD.println()

TypeError: add() missing 1 required positional argument: 'tableProb'

In [ ]:
model = UGraphModel()
print("Build model")
model.add('A')
model.add('B')
model.add('C')
model.add('D')
model.add('E')
model.add('F')
model.connect('A', 'B', 6)
model.connect('A', 'C', 3)

model.connect('B', 'C', 2)
model.connect('B', 'D', 5)

model.connect('C', 'D', 3)
model.connect('C', 'E', 4)

model.connect('D', 'E', 2)
model.connect('D', 'F', 3)

model.connect('E', 'F', 5)

model.println()

print("Disconnect vertex C with remaining vertexes")
model.disconnect('C', 'A')
model.disconnect('C', 'B')
model.disconnect('C', 'D')
model.disconnect('C', 'E')
model.println()

print("Remove vertex A")
model.remove('A')
model.println()

In [12]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

TypeError: add() missing 1 required positional argument: 'tableProb'

In [ ]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

In [16]:
model = UGraphModel()
model.add('A')
model.add('B')
model.add('C')
model.add('D')
model.add('E')
model.add('F')
model.connect('A', 'B', 6)
model.connect('A', 'C', 3)

model.connect('B', 'C', 2)
model.connect('B', 'D', 5)

model.connect('C', 'D', 3)
model.connect('C', 'E', 4)

model.connect('D', 'E', 2)
model.connect('D', 'F', 3)

model.connect('E', 'F', 5)

model.println()

model.disconnect('C', 'A')
model.disconnect('C', 'B')
model.disconnect('C', 'D')
model.disconnect('C', 'E')

model.println()

mst = UGraphModelAlgorithm(model) 
tree = mst.minSpanningTree()
tree.println()

TypeError: add() missing 1 required positional argument: 'tableProb'